<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. 

**Цель исследования** - тобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Построить модель для такого предсказания. Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Мне нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.

Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor, cv, Pool

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('datasets/taxi.csv', index_col=[0], parse_dates = [0])

In [3]:
df

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32
...,...
2018-08-31 23:10:00,32
2018-08-31 23:20:00,24
2018-08-31 23:30:00,27


In [4]:
fig = px.line(df, x = df.index , y = 'num_orders')
fig.show()

Сделаем ресемплинг по 1 дню

In [5]:
df = df.resample('1H').sum()

In [6]:
fig = px.line(df, x = df.index , y = 'num_orders')
fig.show()

## Анализ

### Подготовка данных

Cделаем декомпозицию ряда для выделения тренда, сезонности и остатков

In [7]:
decomposed = seasonal_decompose(df)

In [8]:
seasonal = decomposed.seasonal

In [9]:
trend = decomposed.trend

In [10]:
resid = decomposed.resid

In [11]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Seasonal","Trend", "Resid"))

fig.append_trace(go.Scatter(
    x=seasonal.index,
    y=seasonal,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=trend.index,
    y=trend,
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=resid.index,
    y=resid,
), row=3, col=1)

fig.update_layout(height=800, width=1_600, title_text="Time series decomposition")
fig.show()

Исследуемый ряд не является станционарным в виду наличия сезонности, следовательно меняется среднее значение ряда со врменем.

### Создание новых фичей для обучения

In [12]:
df

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43
...,...
2018-08-31 19:00:00,136
2018-08-31 20:00:00,154
2018-08-31 21:00:00,159


In [13]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()
    
    return data

In [39]:
df_model = make_features(df,3,2)

In [40]:
df_model

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,rolling_mean,lag_3
datetime,,,,,,,,,,
2018-03-01 00:00:00,124,2018,3,1,3,0,NaN,NaN,NaN,NaN
2018-03-01 01:00:00,85,2018,3,1,3,1,124.0,NaN,NaN,NaN
2018-03-01 02:00:00,71,2018,3,1,3,2,85.0,124.0,104.5,NaN
2018-03-01 03:00:00,66,2018,3,1,3,3,71.0,85.0,78.0,124.0
2018-03-01 04:00:00,43,2018,3,1,3,4,66.0,71.0,68.5,85.0
...,...,...,...,...,...,...,...,...,...,...
2018-08-31 19:00:00,136,2018,8,31,4,19,207.0,217.0,212.0,197.0
2018-08-31 20:00:00,154,2018,8,31,4,20,136.0,207.0,171.5,217.0
2018-08-31 21:00:00,159,2018,8,31,4,21,154.0,136.0,145.0,207.0


In [56]:
train, test = train_test_split(df_model, shuffle=False, test_size=0.1)

In [57]:
train = train.dropna()

In [58]:
train

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,rolling_mean,lag_3
datetime,,,,,,,,,,
2018-03-01 03:00:00,66,2018,3,1,3,3,71.0,85.0,78.0,124.0
2018-03-01 04:00:00,43,2018,3,1,3,4,66.0,71.0,68.5,85.0
2018-03-01 05:00:00,6,2018,3,1,3,5,43.0,66.0,54.5,71.0
2018-03-01 06:00:00,12,2018,3,1,3,6,6.0,43.0,24.5,66.0
2018-03-01 07:00:00,15,2018,3,1,3,7,12.0,6.0,9.0,43.0
...,...,...,...,...,...,...,...,...,...,...
2018-08-13 09:00:00,137,2018,8,13,0,9,91.0,39.0,65.0,66.0
2018-08-13 10:00:00,156,2018,8,13,0,10,137.0,91.0,114.0,39.0
2018-08-13 11:00:00,144,2018,8,13,0,11,156.0,137.0,146.5,91.0


In [95]:
X = train.drop(['num_orders','year'], axis = 1)
y = train['num_orders']

## Обучение

### CatboostRegressor

In [96]:
params = {
    'loss_function': 'RMSE',
    'iterations': 5_000,
    'random_seed': 42,
    'learning_rate': 0.001,
    'early_stopping_rounds': 100
}

In [97]:
cv_data = cv(
    params=params,
    pool=Pool(X, label=y),
    fold_count=5, # Разбивка выборки на 5 кусочков
    partition_random_seed=42,
    plot=True, # Никуда без визуализатора
    stratified=True, 
    verbose=False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]



bestTest = 30.25920452
bestIteration = 4999

Training on fold [1/5]



bestTest = 24.26161536
bestIteration = 4999

Training on fold [2/5]



bestTest = 22.8862487
bestIteration = 4999

Training on fold [3/5]



bestTest = 22.1714536
bestIteration = 4999

Training on fold [4/5]



bestTest = 20.51122162
bestIteration = 4998



In [98]:
cv_data

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,86.738271,5.454652,87.203268,1.533074
1,1,86.667665,5.455473,87.132335,1.532243
2,2,86.592567,5.456002,87.056803,1.531198
3,3,86.520178,5.456861,86.984441,1.530592
4,4,86.446923,5.458008,86.911163,1.530170
...,...,...,...,...,...
4995,4995,24.019704,3.742669,22.725079,0.718899
4996,4996,24.019449,3.742305,22.724554,0.718942
4997,4997,24.019070,3.742340,22.723833,0.718720
4998,4998,24.018314,3.742208,22.722871,0.718460


Имеем лучшее значение RMSE в результате проверки модели кросс-валидацией 24.01

### RandomForestRegresssor

In [63]:
best_model = None
best_result = 50

In [64]:
param_grid = {'max_depth': list(range(1, 10)), 'n_estimators': list(range(80,100))}
params = list(ParameterGrid(param_grid))
for param in tqdm(params):
    model = RandomForestRegressor(random_state=42, max_depth=param['max_depth'], n_estimators=param['n_estimators'], n_jobs= -1)
    scores = abs(cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error', error_score='raise', n_jobs= -1))
    final_score = (sum(scores) / len(scores))**0.5  
    if best_result > final_score:
        best_result = final_score
        best_model = model

  0%|          | 0/180 [00:00<?, ?it/s]

In [65]:
print('Best model is:',best_model)
print('Best RMSE metric is:',best_result)

Best model is: RandomForestRegressor(max_depth=9, n_estimators=92, n_jobs=-1, random_state=42)
Best RMSE metric is: 25.647268730054392


### Вывод

- Лучший результат на катбусте 24.574465
- Лучший результат на случайно лесе 25.647268730054392

## Тестирование

In [71]:
test

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,rolling_mean,lag_3
datetime,,,,,,,,,,
2018-08-13 14:00:00,102,2018,8,13,0,14,119.0,92.0,105.5,144.0
2018-08-13 15:00:00,175,2018,8,13,0,15,102.0,119.0,110.5,92.0
2018-08-13 16:00:00,144,2018,8,13,0,16,175.0,102.0,138.5,119.0
2018-08-13 17:00:00,152,2018,8,13,0,17,144.0,175.0,159.5,102.0
2018-08-13 18:00:00,104,2018,8,13,0,18,152.0,144.0,148.0,175.0
...,...,...,...,...,...,...,...,...,...,...
2018-08-31 19:00:00,136,2018,8,31,4,19,207.0,217.0,212.0,197.0
2018-08-31 20:00:00,154,2018,8,31,4,20,136.0,207.0,171.5,217.0
2018-08-31 21:00:00,159,2018,8,31,4,21,154.0,136.0,145.0,207.0


In [99]:
X = test.drop(['num_orders','year'], axis = 1)
y = test['num_orders']

### CatboostRegressor

In [120]:
cbr = CatBoostRegressor(loss_function='RMSE',
                          iterations=5_000,
                          learning_rate=0.001,
                          verbose=False,
                          random_seed = 42,   
)

In [121]:
cbr.fit(X,y,
          plot = True,
          early_stopping_rounds=20
)
print('Model is fitted: ' + str(cbr.is_fitted()))
print('Model params:')
print(cbr.get_params())

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model is fitted: True
Model params:
{'iterations': 10000, 'learning_rate': 0.003, 'loss_function': 'RMSE', 'random_seed': 42, 'verbose': False}


In [122]:
pred = cbr.predict(X)

In [123]:
cbr_rmse = mean_squared_error(y,pred)**0.5

In [124]:
cbr_rmse

12.205610122945144

### RandomForestRegresssor

In [70]:
print('Best model is:',best_model)
print('Best RMSE metric is:',best_result)

Best model is: RandomForestRegressor(max_depth=9, n_estimators=92, n_jobs=-1, random_state=42)
Best RMSE metric is: 25.647268730054392


In [80]:
best_model.fit(X,y)

RandomForestRegressor(max_depth=9, n_estimators=92, n_jobs=-1, random_state=42)

In [81]:
pred = best_model.predict(X)

In [82]:
rf_rmse = mean_squared_error(y,pred)**0.5

In [83]:
rf_rmse

18.777585592887824

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Данные проанализированы
- [ ]  Модель обучена, гиперпараметры подобраны
- [ ]  Качество моделей проверено, выводы сделаны
- [ ]  Значение *RMSE* на тестовой выборке не больше 48